<a href="https://colab.research.google.com/github/partizanos/advanced_image_processing/blob/master/tp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image
from io import BytesIO

response = requests.get("https://raw.githubusercontent.com/partizanos/advanced_image_processing/master/TP/TP4/test.png")
im = Image.open(BytesIO(response.content))
img = np.array(im)


def extract_colors(img):
  R_DIMENSION = img[:, :, 0]
  G_DIMENSION = img[:, :, 1]
  B_DIMENSION = img[:, :, 2]
  return R_DIMENSION, G_DIMENSION, B_DIMENSION


def YCbCr(R_DIMENSION,G_DIMENSION,B_DIMENSION): 
  Y = np.add(np.add(0.299 * R_DIMENSION, 0.587 * G_DIMENSION), 0.114 * B_DIMENSION)
  Cb = - 0.1687 * R_DIMENSION - 0.3313 * G_DIMENSION + 0.5 * B_DIMENSION + 128
  Cr = 0.5 * R_DIMENSION - 0.4187 * G_DIMENSION - 0.0813 * B_DIMENSION + 128
  R_DIMENSION = Y + 1.402 * (Cr - 128)
  G_DIMENSION = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
  B_DIMENSION = Y + 1.772 * (Cb - 128)
  return Y, Cb, Cr, R_DIMENSION, G_DIMENSION, B_DIMENSION

Y, Cb, Cr, R_DIMENSION, G_DIMENSION, B_DIMENSION = YCbCr(*extract_colors(img))


In [52]:
def downsample_image(img, skip):
    return img[::skip,::skip]

d_cb  = downsample_image(Cb, 2)
d_cb  = downsample_image(d_cb, 2)
d_cr  = downsample_image(Cr, 2)
d_cr  = downsample_image(d_cr, 2)
Y[0].shape

(1296,)

In [59]:
import math
M, N, _ = img.shape
max_M = int(math.modf(M/8)[1] *8)
max_N = int(math.modf(N/8)[1] *8)
global_mean = 128
number_octads = int(max_M* max_N / 64)
number_octads, Y.shape, max_M, max_N

(17496, (870, 1296), 864, 1296)

In [63]:
crop = lambda img:img[:max_M, :max_N]
Cb_cropped = crop(Cb)
Cr_cropped = crop(Cr)
Y_cropped = crop(Y)
Y_r = Y_cropped.reshape((number_octads, 8,8)) - global_mean
Cb_r = Cb_cropped.reshape((number_octads, 8,8)) - global_mean
Cr_r = Cr_cropped.reshape((number_octads, 8,8)) - global_mean

array([[-110.228, -108.054, -101.592, -114.293, -115.364, -112.723,
        -114.495, -117.783],
       [-116.196, -114.196, -113.159, -107.877, -105.501, -104.886,
         -98.587,  -94.576],
       [ -86.559,  -90.086,  -98.755, -105.712, -110.957,  -99.185,
        -103.071, -106.957],
       [-110.071, -109.256, -102.74 , -101.968,  -91.256,  -73.576,
         -73.217,  -63.706],
       [ -35.389,  -27.393,  -77.096, -102.054,  -90.185,  -67.408,
         -64.234,  -70.87 ],
       [ -66.989,  -94.114,  -97.342,  -95.168,  -97.228,  -94.131,
         -86.066,  -41.371],
       [  13.539,   26.116,   30.123,   14.737,  -10.28 ,  -22.091,
          22.997,   31.955],
       [  40.673,   33.266,    0.712,  -38.916,  -44.091,   11.39 ,
           5.111,  -28.282]])

In [0]:
# 4. DCT transformation per block: T (u, v)
from scipy import fftpack 
Y_dct = fftpack.dct(Y_r)
Cb_dct = fftpack.dct(Cb_r)
Cr_dct = fftpack.dct(Cr_r)

In [0]:
Z = [
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 28, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
]

T_Y = Y_dct/Z
T_cb = Cb_dct/Z
T_cr = Cr_dct/Z